In [ ]:
-- https://github.com/smpetersgithub/AdvancedSQLPuzzles/tree/main/Advanced%20SQL%20Puzzles
-- But in Snowflake!

In [ ]:
-- Puzzle 1 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS Cart1
    (
    Item  VARCHAR(100) PRIMARY KEY
    )
;

CREATE TEMPORARY TABLE IF NOT EXISTS Cart2
    (
    Item  VARCHAR(100) PRIMARY KEY
    )
;

INSERT INTO Cart1 (Item) VALUES
('Sugar'),('Bread'),('Juice'),('Soda'),('Flour')
;

INSERT INTO Cart2 (Item) VALUES
('Sugar'),('Bread'),('Butter'),('Cheese'),('Fruit')
;

In [ ]:
-- Puzzle 1
SELECT
    *
FROM
    cart1 AS c1
    FULL JOIN cart2 AS c2 ON c2.item=c1.item
;

In [ ]:
-- Puzzle 2 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS Employees
    (
    EmployeeID  INTEGER PRIMARY KEY,
    ManagerID   INTEGER NULL,
    JobTitle    VARCHAR(100) NOT NULL
    )
;

INSERT INTO Employees (EmployeeID, ManagerID, JobTitle)
VALUES
    (1001,NULL,'President')
    ,(2002,1001,'Director')
    ,(3003,1001,'Office Manager')
    ,(4004,2002,'Engineer')
    ,(5005,2002,'Engineer')
    ,(6006,2002,'Engineer')
;

In [ ]:
-- Puzzle 2 (Solution 1)
WITH emp_hierarchy AS
    (
    SELECT
        e.employeeid
        ,e.managerid
        ,e.jobtitle
        ,0 AS depth
    FROM
        employees AS e
    WHERE 1=1
        AND e.managerid IS NULL

    UNION ALL

    SELECT
        e.employeeid
        ,e.managerid
        ,e.jobtitle
        ,eh.depth+1 AS depth
    FROM
        emp_hierarchy AS eh
        JOIN employees AS e ON e.managerid=eh.employeeid
    )

SELECT
    *
FROM
    emp_hierarchy AS eh
ORDER BY
    eh.depth
;

In [ ]:
-- Puzzle 2 (Solution 2)
SELECT
    e.employeeid
    ,e.managerid
    ,e.jobtitle
    ,ARRAY_SIZE(SPLIT(SUBSTR(SYS_CONNECT_BY_PATH(e.jobtitle, '.'), 2, LENGTH(SYS_CONNECT_BY_PATH(e.jobtitle, '.'))), '.'))-1 AS depth
FROM
    employees AS e
    START WITH
        jobtitle = 'President'
    CONNECT BY
        managerid = PRIOR employeeid
ORDER BY
    employeeid
;

In [ ]:
-- Puzzle 3 DDL
CREATE OR REPLACE TEMPORARY TABLE EmployeePayRecords
    (
    EmployeeID  INTEGER,
    FiscalYear  INTEGER,
    StartDate   DATE,
    EndDate     DATE,
    PayRate     DECIMAL(38,2)
    )
;

In [ ]:
-- Snowflake doesn't support SQL Server Check Constraints

ALTER TABLE EmployeePayRecords ADD CONSTRAINT unq_emp_fiscyear UNIQUE (employeeid, fiscalyear);

In [ ]:
-- Puzzle 4 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS Orders
    (
    CustomerID     INTEGER,
    OrderID        INTEGER,
    DeliveryState  VARCHAR(100) NOT NULL,
    Amount         DECIMAL(38,2) NOT NULL
    )
;

INSERT INTO Orders (CustomerID, OrderID, DeliveryState, Amount)
VALUES
    (1001,1,'CA',340)
    ,(1001,2,'TX',950)
    ,(1001,3,'TX',670)
    ,(1001,4,'TX',860)
    ,(2002,5,'WA',320)
    ,(3003,6,'CA',650)
    ,(3003,7,'CA',830)
    ,(4004,8,'TX',120)
;

In [ ]:
-- Puzzle 4
SELECT
    *
FROM
    orders AS o
WHERE 1=1
    AND o.deliverystate='TX'
    AND EXISTS(
                SELECT
                    1
                FROM
                    orders AS o2
                WHERE 1=1
                    AND o2.customerid=o.customerid
                    AND o2.deliverystate='CA'
                )
;

In [ ]:
-- Puzzle 5 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS PhoneDirectory
    (
    CustomerID   INTEGER,
    Type       VARCHAR(100),
    PhoneNumber  VARCHAR(12) NOT NULL
    )
;

INSERT INTO PhoneDirectory (CustomerID, Type, PhoneNumber)
VALUES
    (1001,'Cellular','555-897-5421')
    ,(1001,'Work','555-897-6542')
    ,(1001,'Home','555-698-9874')
    ,(2002,'Cellular','555-963-6544')
    ,(2002,'Work','555-812-9856')
    ,(3003,'Cellular','555-987-6541')
;

In [ ]:
-- Puzzle 5 (Solution 1 - Clean)
SELECT
    p.customerid
    ,p."'Cellular'" AS cellular
    ,p."'Work'" AS work
    ,p."'Home'" AS home
FROM
    phonedirectory AS pd
    PIVOT (MAX(phonenumber) FOR type IN (ANY ORDER BY type)) AS p
;

In [ ]:
-- Puzzle 5 (Solution 2 - Hacky)
SELECT
    pd.customerid
    ,MAX(IFF(pd.type='Cellular', pd.phonenumber, NULL)) AS cellular
    ,MAX(IFF(pd.type='Work', pd.phonenumber, NULL)) AS work
    ,MAX(IFF(pd.type='Home', pd.phonenumber, NULL)) AS home
FROM
    phonedirectory AS pd
GROUP BY ALL
ORDER BY
    pd.customerid
;

In [ ]:
-- Puzzle 6 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS WorkflowSteps
    (
    Workflow        VARCHAR(100),
    StepNumber      INTEGER,
    CompletionDate  DATE NULL
    )
;

INSERT INTO WorkflowSteps (Workflow, StepNumber, CompletionDate)
VALUES
    ('Alpha',1,'7/2/2018')
    ,('Alpha',2,'7/2/2018')
    ,('Alpha',3,'7/1/2018')
    ,('Bravo',1,'6/25/2018')
    ,('Bravo',2,NULL)
    ,('Bravo',3,'6/27/2018')
    ,('Charlie',1,NULL)
    ,('Charlie',2,'7/1/2018')
;

In [ ]:
-- Puzzle 6 (Solution 1)
SELECT
    ws.workflow
FROM
    workflowsteps AS ws
WHERE 1=1
    AND completiondate IS NULL
;

In [ ]:
-- Puzzle 6 (Solution 2 - Bonus)
SELECT
    ws.workflow
FROM
    workflowsteps AS ws
GROUP BY
    ws.workflow
HAVING
    COUNT(*) <> COUNT(ws.completiondate)
;

In [ ]:
-- Puzzle 7 DDL
CREATE OR REPLACE TEMPORARY TABLE Candidates
    (
    CandidateID  INTEGER,
    Occupation   VARCHAR(100)
    )
;

INSERT INTO Candidates (CandidateID, Occupation)
VALUES
    (1001,'Geologist')
    ,(1001,'Astrogator')
    ,(1001,'Biochemist')
    ,(1001,'Technician')
    ,(2002,'Surgeon')
    ,(2002,'Machinist')
    ,(2002,'Geologist')
    ,(3003,'Geologist')
    ,(3003,'Astrogator')
    ,(4004,'Selenologist')
;

CREATE OR REPLACE TEMPORARY TABLE Requirements
    (
    Requirement  VARCHAR(100)
    )
;

INSERT INTO Requirements (Requirement)
    VALUES
    ('Geologist')
    ,('Astrogator')
    ,('Technician')
;

In [ ]:
-- Puzzle 7
;WITH count_requirements AS
    (
    SELECT
        COUNT(*) AS ct
    FROM
        requirements AS r
    )
    
SELECT
    c.candidateid
    ,cr.ct
FROM
    candidates AS c
    JOIN requirements AS r ON r.requirement=c.occupation
    CROSS JOIN count_requirements AS cr
GROUP BY ALL
HAVING
    COUNT(*)=cr.ct
;

In [ ]:
-- Puzzle 8 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS WorkflowCases
    (
    Workflow  VARCHAR(100),
    Case1     INTEGER NOT NULL DEFAULT 0,
    Case2     INTEGER NOT NULL DEFAULT 0,
    Case3     INTEGER NOT NULL DEFAULT 0
    )
;

INSERT INTO WorkflowCases (Workflow, Case1, Case2, Case3)
VALUES
    ('Alpha',0,0,0)
    ,('Bravo',0,1,1)
    ,('Charlie',1,0,0)
    ,('Delta',0,0,0)
;

In [ ]:
-- Puzzle 8 (Solution 1 - Clean)
SELECT
    up.workflow
    ,SUM(up.passed) AS passed
FROM
    workflowcases AS wc
    UNPIVOT(passed FOR case IN (Case1, Case2, Case3)) AS up
GROUP BY ALL
;

In [ ]:
-- Puzzle 8 (Solution 2 - Hacky)
SELECT
    wc.workflow
    ,wc.case1+wc.case2+wc.case3 AS passed
FROM
    workflowcases AS wc
;

In [ ]:
-- Puzzle 9 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS Employees
    (
    EmployeeID  INTEGER,
    License     VARCHAR(100)
    )
;

INSERT INTO Employees (EmployeeID, License)
VALUES
    (1001,'Class A')
    ,(1001,'Class B')
    ,(1001,'Class C')
    ,(2002,'Class A')
    ,(2002,'Class B')
    ,(2002,'Class C')
    ,(3003,'Class A')
    ,(3003,'Class D')
    ,(4004,'Class A')
    ,(4004,'Class B')
    ,(4004,'Class D')
    ,(5005,'Class A')
    ,(5005,'Class B')
    ,(5005,'Class D')
;

In [ ]:
-- Puzzle 9
;WITH employees_enhanced AS
    (
    SELECT
        *
        ,COUNT(*) OVER(PARTITION BY e.employeeid) AS ct_employee_licenses
    FROM
        employees AS e
    )
    
SELECT DISTINCT
    e.employeeid
    ,e2.employeeid
    ,COUNT(*) OVER(PARTITION BY e.employeeid, e2.employeeid) AS ct_combo_licenses
FROM
    employees_enhanced AS e
    JOIN employees_enhanced AS e2 ON e2.employeeid<>e.employeeid AND e2.license=e.license
WHERE 1=1
    AND e.ct_employee_licenses=e2.ct_employee_licenses
QUALIFY
    e.ct_employee_licenses=ct_combo_licenses
ORDER BY
    e.employeeid
    ,e2.employeeid
;

In [ ]:
-- Puzzle 10 DDL
CREATE TEMPORARY TABLE IF NOT EXISTS SampleData
    (
    IntegerValue  INTEGER NOT NULL
    )
;

INSERT INTO SampleData (IntegerValue)
VALUES
    (5),(6),(10),(10),(13),(14),(17),(20),(81),(90),(76)
;

In [ ]:
-- Puzzle 10
SELECT
    AVG(sd.integervalue)
    ,MEDIAN(sd.integervalue)
    ,MODE(sd.integervalue)
    ,MAX(sd.integervalue)-MIN(sd.integervalue)
FROM
    sampledata AS sd
;